<a href="https://colab.research.google.com/github/dangeunii/AI_code/blob/main/%ED%8A%B8%EB%A6%AC%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0_%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **교차검증과 그리드 서치**

# 교차검증

 훈련세트, 테스트 세트 외에도 검증세트(validation set)가 필요
  - 훈련세트는 훈련을 위해서 사용, 테스트 세트는 실전을 위해서 한번만 사용
  - 머신러닝 모델을 최적화하는 과정에서 검증을 위한 데이터 세트가 필요함
  - 일반적으로 훈련세트에서 검증세트를 추출



In [16]:
## 데이터 준비
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [17]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state = 42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [18]:
## 훈련세트를 검증세트와 훈련세트로 나누어줌
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [21]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
print(dt.score(test_input, test_target))

0.9971133028626413
0.864423076923077
0.8569230769230769


# 교차 검증

훈련데이터 세트가 클수록 성능이 높아짐

  - 검증세트를 최대한 활용하도록 함 -> 교차검증
  - k-fold cross validation : 훈련세트를 몇 부분(k)으로 나누어 모델 훈련을 수행
  - k번 모델 훈련을 수행한 검증 점수의 평균으로 검증점수 결정
  - 5-폴드 교차검정(데이터세트 80% 사용), 10-폴드 교차검정(데이터세트 90% 사용)

  - sklearn.model_selection.cross_validate()
    : 모델과 데이터세트에 대한 교차 검증 수행
 
    - 기본 설정 : 5- 폴드 교차 검증
    - 출력값 : fit_time, score_time, test_score
    - 훈련세트를 섞지 않기에 결과가 치우칠 우려있음 -> 분할기(splitter) 지정 필요

  - sklearn,model_selection.StratifiedKFold()
    : 계층적 k-폴드 분할에 사용

    - n_splits : 분리할 데이터 셋의 개수, k-폴드의 k값, 기본값(5)
    - shuffle : 배치로 분활하기 전에 각 클래스의 샘플을 섞음
    - test_size : 테스트 세트의 비율 지정
    - train_size :  훈련세트의 비율 지정

In [22]:
## 교차 검증 수행 후 결과

from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)


{'fit_time': array([0.00774789, 0.00688481, 0.00736117, 0.00715852, 0.00678658]), 'score_time': array([0.00066566, 0.00058222, 0.00063181, 0.00058985, 0.00056767]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [24]:
## 교차검증의 평균 출력
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [29]:
## 데이터 세트를 섞어서 10-폴드로 수행
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [30]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 모델 최적화 - 하이퍼 파라미터 튜닝



*   하이퍼 파라미터 튜닝
    - 하이퍼 파라미터 : 머신러닝 모델이 학습하는 모델 파라미터에 사용자가 지정하는 파라미터
    - 하이퍼 파라미터의 최적화를 위해 모델의 매개변수를 변경하면서 훈련 및 교차 검증 수행
    - 결정트리의 경우 max_depth, main_samples_split 값 변경하면서 최적의 값을 찾을 수 있다.
*   그리드 서치(grid search)
   - 매개변수의 조합을 반복적으로 적용하면서 훈련 및 교차검증을 수행하면서 결과 출력
   - GridSearchCV 클래스로 하이퍼 파라미터 탐색과 교차검증 수행(for문 대신)



# 랜덤 서치(random search)

- 모델 최적화를 위한 매개변수값이 너무 많을 때
- 매개변수 값 목록 전달 -> 매개변수 샘플링을 위한 확률 분포 객체 전달


  - scipy.stats.uniform() : 균등하기 연속적인 실수 값 출력 / uniforn.rvs() : 출력 개수 지정
  - scipy.stats.randint() : 균등하게 임의의 정수 값 출력 / randint.rvs() : 출력 개수 지정
  - sklearn.model_selection.RandomizedSearchCV()
    - estimator : 모델 객체 지정
    - param_distributions : 하이퍼파라미터 목록
    - cv : 교차 검증 폴드 수
    - n_iter : 파라미터 검색 횟수
    - [결과] cv_results_ : 파라미터 조합별 결과 조회
    - [결과] best_params_ : 가장 좋은 성능을 낸 parameter 조합 조회
    - [결과] best_estimator_ : 가장 좋은 성능을 낸 모델 반환 

In [31]:
from scipy.stats import uniform, randint

rgen = randint(0,10)
rgen.rvs(10)

array([3, 2, 2, 3, 2, 6, 6, 9, 7, 1])

In [32]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98,  97, 107, 109,  89, 106,  91, 106, 109,  88]))

In [33]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.43550888, 0.3538995 , 0.83852249, 0.55993858, 0.50924591,
       0.88324213, 0.57012634, 0.66682773, 0.09829837, 0.79574611])

In [34]:
## 매개변수 값 목록 전달 -> 매개변수 샘플링을 위한 확률분포 객체 전달

params = {
    'min_impurity_decrease': uniform(0.0001,0.001),
    'max_depth' : randint(20,50),
    'min_samples_split' : randint(2,25),
    'min_samples_leaf' : randint(1,25), 
}

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state =42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82fa8e2d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82fa8e910>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82fc5c950>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82fa8e510>},
                   random_state=42)

In [35]:
## 결과 확인
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [36]:
## 최적화 모델 평가
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


# 확인 문제

위의 RandomizedSearchCV 실습에서 DecisionTreeClassifier 클래스에 splitter ='random' 매개변수를 추가하고 다시 훈련해보세요.
splitter 매개변수의 기본값은 'best'로 각 노드에서 최선의 분할을 찾습니다.
'random'이면 무작위로 분할한 다음 가장 좋은 것을 고릅니다.
테스트 세트의 성능이 어떻게 변했는지 확인하시오.

In [37]:
## 매개변수 값 목록 전달/ splitter 추가

gs = RandomizedSearchCV(DecisionTreeClassifier(splitter = 'random',random_state =42), params,
                        n_iter=100, n_jobs=-1, random_state=42)

gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82fa8e2d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82fa8e910>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82fc5c950>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82fa8e510>},
                   random_state=42)

In [38]:
## 결과 확인
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [39]:
## 최적화 모델 평가
## 성능이 다소 하락함
print(np.max(gs.cv_results_['mean_test_score']))

0.8458726956392981
